In [13]:
# This script is used for testing all adaptations and changes made

In [37]:
import pandas as pd
import numpy as np

from scipy import linalg
from scipy import stats

In [38]:
# Load data for change detection as a pandas data frame.
infys_df =  pd.read_csv("../data/iMAD/iMAD_Data_13.csv") #need to change the dataset 
infys_df.head()

Unnamed: 0  Cycle  Cluster_ID     AvgDbh  AvgCrownDiameter  AvgCrownHeight  \
0           1      1       77652  16.103571          3.937736        3.367925   
1           2      1       28451  10.380952          2.209524        0.000000   
2           3      1       55242  14.996429          4.439683        4.557937   
3           4      1       20795  11.600000          3.169565        4.286957   
4           5      1       53236  12.194444          4.886047        5.753488   

   AvgCrownArea  SpeciesCount  AvgTreeHeight  TreeCount         J           X  \
0     15.409993             8       8.555357         56  0.440017  -92.433389   
1      3.956920             2       2.909524         21  0.334649  -99.546778   
2     17.199388             5       5.458333         84  0.505430 -104.218713   
3      8.243627             5       4.493548         31  0.770242 -108.957083   
4     20.022403             6       7.605556         54  0.841231  -98.775463   

           Y  
0  15.588935  
1  26.072916  
2  20.403546  
3  27.390907  
4  20.791278

In [39]:
# Create new ids by concatenating 'Conglomerado' and 'Sitio'.
    ## don't think I need to do that
    ## Use existing ID of Metadataset called Cluster_ID -> should be able to leave that one out


# As we want to compare cycle 1 vs cycle 2 lets separate them.   
## Stays the same for long dataformat
## might figure out how to 
infys_df_c1 = infys_df[infys_df['Cycle'] == 1]
infys_df_c2 = infys_df[infys_df['Cycle'] == 3]

In [40]:
# Find intersection of new ID variable: 'cng_sit', so obs. that are present in both cycles.
s1 = infys_df_c1['Cluster_ID']
s2 = infys_df_c2['Cluster_ID']
cluster_id_inter = pd.Series(list(set(s1).intersection(set(s2))))
## might have already done this in R by using my filter(muestreado1 == 1 & muestreado2 == 1)



In [41]:
# Filter out obs. that dont appear in cng_sit_inter.
infys_df_c1 = infys_df_c1[infys_df_c1['Cluster_ID'].isin(cluster_id_inter)]
infys_df_c2 = infys_df_c2[infys_df_c2['Cluster_ID'].isin(cluster_id_inter)]

In [42]:
# Select only variables to be utilized in change detection.
vars = ["SpeciesCount", "TreeCount", "J", "AvgTreeHeight", "AvgDbh", "AvgCrownDiameter", "AvgCrownHeight", "AvgCrownArea"]
nvars = len(vars)
infys_df_c1_c = infys_df_c1[infys_df_c1.columns.intersection(vars)]
infys_df_c2_c = infys_df_c2[infys_df_c2.columns.intersection(vars)]

c1_c_nans = infys_df_c1_c.isna().any(axis=1).to_numpy()
c2_c_nans = infys_df_c2_c.isna().any(axis=1).to_numpy()

In [43]:
# Create change data matrix, missing values mask and initial change weights.
dm = np.zeros((2 * nvars, infys_df_c1_c.shape[0]))
dm[0:nvars] = np.transpose(infys_df_c1_c.to_numpy())
dm[nvars:] = np.transpose(infys_df_c2_c.to_numpy())

nodataidx = c1_c_nans | c2_c_nans
gooddataidx = nodataidx == False
dm = dm[:, gooddataidx]
ngood = np.sum(gooddataidx)

In [44]:
# Change detection iterations.

# iteration of MAD   
wt = np.ones(int(ngood))
delta = 1.0
oldrho = np.zeros(nvars)
iter = 0
 
while (delta > 0.01) and (iter < 35): 
    print(iter)
    # Weighted covariance matrices and means.
    sumw = np.sum(wt)
    means = np.average(dm,axis=1, weights=wt)
    dmc = dm - means[:,np.newaxis]
    dmc = np.multiply(dmc,np.sqrt(wt))
    sigma = np.dot(dmc,dmc.T)/sumw
   
    s11 = sigma[0:nvars, 0:nvars]
    s22 = sigma[nvars:, nvars:]
    s12 = sigma[0:nvars, nvars:]
    s21 = sigma[nvars:, 0:nvars]
    
    # Solution of generalized eigenproblems.
    aux_1 = linalg.solve(s22, s21)
    lama, a = linalg.eig(np.dot(s12, aux_1), s11)
    aux_2 = linalg.solve(s11, s12)
    lamb, b = linalg.eig(np.dot(s21, aux_2), s22)
    
    
    # Sort a.  
    idx = np.argsort(lama)
    a = a[:, idx]
    
    # Sort b.        
    idx = np.argsort(lamb)
    b = b[:, idx]
    
    # Canonical correlations.        
    rho = np.sqrt(np.real(lamb[idx]))
    
    
    # Normalize dispersions.  
    tmp1 = np.dot(np.dot(a.T,s11), a)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    a = np.multiply(a, tmp3)
    b = np.mat(b)
    tmp1 = np.dot(np.dot(b.T,s22), b)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    b = np.multiply(b, tmp3)
        
    # Assure positive correlation
    tmp = np.diag(np.dot(np.dot(a.T,s12), b))
    b = np.dot(b,np.diag(tmp / np.abs(tmp)))

    # Canonical and MAD variates
    U = np.dot(a.T , (dm[0:nvars, :] - means[0:nvars, np.newaxis]))    
    V = np.dot(b.T , (dm[nvars:, :] - means[nvars:, np.newaxis]))          
    MAD = U - V  
    
            
    # New weights.        
    var_mad = np.tile(np.mat(2 * (1 - rho)).T, (1, ngood))    
    chisqr = np.sum(np.multiply(MAD, MAD) / var_mad, 0)
    wt = np.squeeze(1 - np.array(stats.chi2._cdf(chisqr, nvars)))
    
        
    # Continue iteration.        
    delta = np.sum(np.abs(rho - oldrho))
    oldrho = rho
    iter += 1
    
# reshape to original image size, by including nodata pixels   
# actually reshaping to image size plus one to add chi-squared values
MADout = np.zeros((int(nvars+1), infys_df_c1_c.shape[0]))
MADout[0:nvars, gooddataidx] = MAD
MADout[nvars, gooddataidx] = chisqr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [45]:
# check whether adding chi-squared worked -> should be 9 rows (or in pyhton count: 0-8)
MADout_df = pd.DataFrame(MADout)
MADout_df.head(10)

0             1             2             3             4      \
0  5.437405e+00  2.405378e+00  1.327816e+00 -4.552112e+00  1.372696e-01   
1  9.626554e-01 -7.247054e-01  1.033827e+00  1.050210e+00  8.542376e-01   
2 -1.032079e+01 -2.361721e+00 -1.695157e+00 -9.671645e-01 -1.540354e+00   
3  3.134562e+00  1.303670e+00  4.536299e-01 -1.187180e+00  1.648971e+00   
4  3.052105e+00  1.054188e+00  1.368149e+00 -3.395432e+00  7.339530e-01   
5 -3.241667e+00 -1.112736e+00 -6.567736e-01  2.059342e+00 -9.990935e-02   
6  1.523143e-01  2.914294e-01  6.170532e-01  2.882223e-01 -3.698032e-01   
7 -1.728942e+00  6.137319e-01  7.569257e-01  6.885598e-01 -6.084994e-01   
8  1.901730e+13  2.791983e+12  5.451926e+12  3.419309e+12  2.993235e+12   

          5      6             7             8             9      ...  \
0 -9.466198e-01    0.0 -1.066795e+00  3.098533e+00 -1.603075e+00  ...   
1 -2.132211e+00    0.0  6.255356e-01 -1.267391e+00 -6.126498e-01  ...   
2  2.305062e+00    0.0  3.474515e-01  8.033283e-01  1.211310e+00  ...   
3  1.381702e+00    0.0 -4.029347e-01  6.639379e-01 -3.188096e-01  ...   
4  1.529025e+00    0.0 -8.899245e-01  1.338909e+00 -8.785407e-01  ...   
5 -3.018230e-01    0.0  5.076562e-01 -7.793796e-01  3.916627e-01  ...   
6  8.762953e-01    0.0 -2.858156e-01  6.303132e-01 -3.905091e-02  ...   
7  2.519553e-01    0.0 -3.380572e-01  5.193926e-01 -5.827170e-01  ...   
8  4.111345e+12    0.0  1.116508e+12  3.623543e+12  2.147336e+12  ...   

          10019         10020         10021         10022         10023  \
0  9.357227e+00 -5.301276e+00 -4.941792e-01  7.528185e+00 -2.320030e+00   
1 -1.771581e+00 -4.384359e-01  1.774590e+00 -1.718880e+00 -1.124911e+00   
2 -6.306810e+00  5.993173e+00 -2.992284e+00 -4.210023e+00  4.674634e+00   
3  5.571334e+00 -2.379747e+00  1.101107e+00  2.562464e+00 -1.321008e+00   
4  5.664258e+00 -2.464416e+00  1.281144e+00  2.415249e+00 -9.542230e-01   
5 -3.219931e+00  1.536313e+00  7.027491e-02 -2.876604e+00  3.983227e-01   
6  7.966533e-01 -1.074671e+00 -4.955892e-02  7.794359e-01 -6.555390e-01   
7 -1.750283e+00 -8.792048e-01  1.221502e-02 -1.208675e-01 -7.985596e-01   
8  2.244093e+13  1.046734e+13  1.300249e+10  3.087954e+12  6.094487e+12   

          10024         10025         10026         10027         10028  
0 -4.775046e+00  2.076104e+00  2.302529e+00  4.596856e+00  5.927396e+00  
1 -1.424104e+00 -1.955404e+00  1.036951e+00  1.229038e+00  1.560425e-01  
2 -8.866290e-02  4.311729e+00  1.835648e+00 -6.922709e+00 -9.734255e-01  
3  1.391240e+00  2.216319e+00 -2.809122e+00  2.759581e+00  2.778459e+00  
4 -7.450945e-01  1.812253e+00  4.429755e-01  2.246209e+00  3.800367e+00  
5  3.212171e-01  1.095070e+00 -7.435969e-02 -1.042254e+00 -1.799952e+00  
6 -9.567884e-01  7.020531e-01 -1.217793e+00  1.694075e-02 -9.412747e-02  
7 -1.916598e+00 -4.250452e-01 -1.032355e+00  7.323212e-02  7.538249e-01  
8  2.756189e+13  3.528647e+12  1.387936e+13  4.356231e+10  3.646791e+12  

[9 rows x 10029 columns]

In [46]:
# increase dimension of transposed dataframe by adding on column called chi_squared
vars_with_chi_squared = vars + ["chi_squared"]

# transpose matrix 
MADout = np.transpose(MADout).copy()
MADout_df = pd.DataFrame(MADout, columns=vars_with_chi_squared)
# should now have 9 columns and the last one should be chi_squared
MADout_df.head(10)

SpeciesCount  TreeCount          J  AvgTreeHeight    AvgDbh  \
0      5.437405   0.962655 -10.320786       3.134562  3.052105   
1      2.405378  -0.724705  -2.361721       1.303670  1.054188   
2      1.327816   1.033827  -1.695157       0.453630  1.368149   
3     -4.552112   1.050210  -0.967165      -1.187180 -3.395432   
4      0.137270   0.854238  -1.540354       1.648971  0.733953   
5     -0.946620  -2.132211   2.305062       1.381702  1.529025   
6      0.000000   0.000000   0.000000       0.000000  0.000000   
7     -1.066795   0.625536   0.347451      -0.402935 -0.889924   
8      3.098533  -1.267391   0.803328       0.663938  1.338909   
9     -1.603075  -0.612650   1.211310      -0.318810 -0.878541   

   AvgCrownDiameter  AvgCrownHeight  AvgCrownArea   chi_squared  
0         -3.241667        0.152314     -1.728942  1.901730e+13  
1         -1.112736        0.291429      0.613732  2.791983e+12  
2         -0.656774        0.617053      0.756926  5.451926e+12  
3          2.059342        0.288222      0.688560  3.419309e+12  
4         -0.099909       -0.369803     -0.608499  2.993235e+12  
5         -0.301823        0.876295      0.251955  4.111345e+12  
6          0.000000        0.000000      0.000000  0.000000e+00  
7          0.507656       -0.285816     -0.338057  1.116508e+12  
8         -0.779380        0.630313      0.519393  3.623543e+12  
9          0.391663       -0.039051     -0.582717  2.147336e+12

In [47]:
# add coordinates
MADout_df["X"] = infys_df_c1["X"]
MADout_df["Y"] = infys_df_c1["Y"]
MADout_df.head(10)

SpeciesCount  TreeCount          J  AvgTreeHeight    AvgDbh  \
0      5.437405   0.962655 -10.320786       3.134562  3.052105   
1      2.405378  -0.724705  -2.361721       1.303670  1.054188   
2      1.327816   1.033827  -1.695157       0.453630  1.368149   
3     -4.552112   1.050210  -0.967165      -1.187180 -3.395432   
4      0.137270   0.854238  -1.540354       1.648971  0.733953   
5     -0.946620  -2.132211   2.305062       1.381702  1.529025   
6      0.000000   0.000000   0.000000       0.000000  0.000000   
7     -1.066795   0.625536   0.347451      -0.402935 -0.889924   
8      3.098533  -1.267391   0.803328       0.663938  1.338909   
9     -1.603075  -0.612650   1.211310      -0.318810 -0.878541   

   AvgCrownDiameter  AvgCrownHeight  AvgCrownArea   chi_squared           X  \
0         -3.241667        0.152314     -1.728942  1.901730e+13  -92.433389   
1         -1.112736        0.291429      0.613732  2.791983e+12  -99.546778   
2         -0.656774        0.617053      0.756926  5.451926e+12 -104.218713   
3          2.059342        0.288222      0.688560  3.419309e+12 -108.957083   
4         -0.099909       -0.369803     -0.608499  2.993235e+12  -98.775463   
5         -0.301823        0.876295      0.251955  4.111345e+12 -103.579314   
6          0.000000        0.000000      0.000000  0.000000e+00 -101.453185   
7          0.507656       -0.285816     -0.338057  1.116508e+12 -108.212148   
8         -0.779380        0.630313      0.519393  3.623543e+12 -103.285564   
9          0.391663       -0.039051     -0.582717  2.147336e+12  -98.962972   

           Y  
0  15.588935  
1  26.072916  
2  20.403546  
3  27.390907  
4  20.791278  
5  23.316000  
6  26.362889  
7  28.779231  
8  23.454722  
9  16.723870

In [48]:
# create .csv
MADout_df.to_csv('iMAD_results_13.csv', index=False)

In [ ]:
#### ENDE ####

In [54]:
## Beta

MADout_df["Y"] = infys_df_c1["std_chg"]
np.std(madout(lstone))
madout(lastone <2*np.std)=2


# Add chi squared variable to table

CHIsqr_df = pd.DataFrame(chisqr)


chisqr = np.transpose(chisqr).copy()
CHI_df = pd.DataFrame(chisqr)

MADout_df['CHI'] = CHI_df



# Assuming chisqr is a 1D numpy array with length equal to the number of rows in MADout
# We ensure chisqr is reshaped as a 2D column vector if it isn't already
chisqr_column = chisqr.reshape(-1, 1)

# Append chisqr as a new column to MADout
MADout = np.concatenate((MADout, chisqr_column), axis=1)




MADout_df.head()



SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (54363561.py, line 5)